In [1]:
import dill
from fastai.tabular.all import *
from fastai.vision.all import *
from fastai.vision.utils import get_image_files

In [2]:
# import model
learn = load_learner(Path("/blue/hulcr/gmarais/Beetle_classifier/Models") / 'beetle_classifier.pkl', cpu=False, pickle_module=dill)
print(learn.dls.vocab) # print all possible classes of model

['Coccotypes_dactyliperda', 'Hylesinus_varius', 'Phloeosinus_dentatus', 'Pityophthorus_juglandis', 'Platypus_cylindrus', 'Pycnarthrum_hispidium', 'Scolotodes_schwarzi', 'Xyleborus_affinis', 'Xylosandrus_compactus', 'Xylosandrus_crassiusculus']


In [ ]:
# import data
dataset_path=r"/blue/hulcr/gmarais/Beetle_data/selected_images"
files = get_image_files(path=dataset_path, recurse=True, folders=('test_data'))
test_dl = learn.dls.test_dl(files, with_labels=True)

In [3]:
preds, targets = learn.get_preds(dl=test_dl)
val_out = learn.validate(dl=test_dl)
print(" Loss: "+str(val_out[0])+"\n",
      "Error Rate: "+str(val_out[1])+"\n",
      "Accuracy: "+str(val_out[2])+"\n",
      "Top k(5) Accuracy: "+str(val_out[3])+"\n")

 Loss: 0.6081917881965637
 Error Rate: 0.011601585894823074
 Accuracy: 0.9883984327316284
 Top k(5) Accuracy: 1.0



In [ ]:
# this function only describes how much a singular value in al ist stands out.
# if all values in the lsit are high or low this is 1
# the smaller the proportiopn of number of disimilar vlaues are to other more similar values the lower this number
# the larger the gap between the dissimilar numbers and the simialr number the smaller this number
# only able to interpret probabilities or values between 0 and 1
# this function outputs an estimate an inverse of the classification confidence based on the probabilities of all the classes.
# the wedge threshold splits the data on a threshold with a magnitude of a positive int to force a ledge/peak in the data
def unkown_prob_calc(probs, wedge_threshold, wedge_magnitude=1, wedge=True):
    if wedge:
        probs = np.where(probs<=wedge_threshold , probs**(2*wedge_magnitude), probs)
        probs = np.where(probs>=wedge_threshold , probs**(1/(2*wedge_magnitude)), probs)
    diff_matrix = np.abs(probs[:, np.newaxis] - probs)
    diff_matrix_sum = np.sum(diff_matrix)
    probs_sum = np.sum(probs)
    class_val = (diff_matrix_sum/probs_sum)
    max_class_val = ((len(probs)-1)*2)
    kown_prob = class_val/max_class_val
    unknown_prob = 1-kown_prob
    
    return(unknown_prob)


unkown_prob_calc(probs=pred, wedge_threshold=0.9, wedge_magnitude=1, wedge=True)

In [ ]:
# build test/apply function that also adds the unknown calss based on threshold
# threshold should be trained with validation dataset
# pipeline:
    # train sweep with aprameter tuning
    # retrain single model with best parameters
    # save model
    # train threshold for unkown class with validation dataset (ROC curve)
    # test model with unkown class on test data using 5-fold cross validation (turn single model train script into CV script that also does testing on test data)

In [ ]:
# FIX GITHUB OF BEETLE CLASSIFIER WITH DOWNLAOD

# FIRST ADD TESTING PART AND THRESHOLD TRAINING TO TRAIN SCRIPT OF SINGLE MODEL
# ADD CV TO TRAINING SINGLE SCRIPT
# UPLOAD MODEL TO HUGGINGFACE
# FINISH APPLY SCRIPT
# ADD APPLY PIPELINE TO GRADIO 
# PARAMETER TUNING SCRIPT AND RUN ON HIPERGATOR
# RUN EXPERIMENTS 